#  Scipyによる最適化 -第2回-

Scientific computing tools or Python

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

SciPyは，Pythonによる科学技術計算のためのオープンソースソフトウェアを利用可能な仕組みである。

Scipyは，NumPyパッケージを内部で用いている。具体的には，NumPyで定義された基本的なデータ構造（配列や行列）や，基本的な演算を用いている。また，図示するためにはMatplotlibを用いている。



# 制約なし最適化（多変数スカラー関数） - `minimize`


`minimize`関数は，`scipy.optimize`で利用可能な，多変数スカラー関数の制約なし最適化・制約付き最適化のためのインターフェイスである。

最適化問題の例として，次の$N変数のRosenbrock関数の最小化問題を取り上げる。　

$$
f(x)=\sum_{i=1}^{N-1}　100\left(x_{i+1}-x_i^2 \right)^2 + \left( 1-x_i\right)^2
$$

この関数の最小解は直感で得ることができ，明らかに，$x_i=1$である。

Rosenbrock関数は，よく用いられる関数である。したがって，`scipy.optimize`に既に定義されており，ユーザがあらためて定義する必要はない。ここでは，この関数を用いて，目的関数の定義，ヤコビ行列の定義，ヘッセ行列の定義の方法を述べる。

ヤコビ行列とヘッセ行列については下記の文書を参照すること。

安部恵介，関数の勾配とヘッセ行列，九州産業大学理工学部講義資料

http://www.is.kyusan-u.ac.jp/~abe/optprba153.pdf



## ニュートンCG法

多変数スカラー関数の最小化問題を解くアルゴリズムには様々なものがある。ここではそれらのうち，ニュートンCG法を扱う。ニュートンCG法は，ニュートン法を修正したものである。この方法では，局所的ヘッセ行列の逆行列を求めるために，CGアルゴリズムを用いる。

ニュートン法は，関数$f(x)$を二次関数で局所的に近似する方法に基づいている。

$$
f(x) \approx f(x_0) + \nabla f(x_0)\cdot \left(x-x_0 \right)+\frac{1}{2}\left(x-x_0\right)^{\top}H\left(x_0\right)\left(x-x_0\right),
$$

ここで，$H(x_0)$はヘッセ行列である。$H(x_0)$は$\nabla^2 f(x_0)$とも表される。ヘッセ行列が正定値であれば，二次形式の勾配を0とすることで関数$f(x)$の局所最小解を求めることができる。この局所最適解は，次の式で与えられる。

$$
x_{\text{opt}} = x_0 - H^{-1} \nabla f.
$$

これから，ヘッセ行列の逆行列は，CG法を用いて計算される。この方法で，Rosenbrock関数の局所最小解を求める方法を示す。

ニュートンCG法はヘッセ行列を用いるので，ヘッセ行列の計算方法を与える必要がある。ここで注意することは，ヘッセ行列自体は求める必要はなく，ヘッセ行列と任意のベクトルとの積を評価することができればよい。したがって，ヘッセ行列を与える方法を指定してもよいし，ヘッセ行列と任意のベクトルとの積を評価する方法を指定してもよい。

### ヘッセ行列を与える方法

Rosenbrock関数のヘッセ行列は，次の式で与えられる。

$$
H_{ij}=\frac{\partial^2 f}{\partial x_i \partial x_j} = 200
\left(
\delta_{i,j} - 2x_{i-1} \delta_{i-1,j}  \right)
-400 x_i \left(\delta_{i+1,j}-2x_i\delta_{i,j} \right) 
-400 \delta_{i,j} \left( x_{i+1}-x_i^2 \right)+2 \delta_{i,j}
$$
$$
= \left( 202+1200x_i^2-400x_{i+1}\right)\delta_{i,j} - 400x_i \delta_{i+1,j} - 400x_{i-1} \delta_{i-1,j}
$$


--------
## 課題1

$N=3$のRosenbrock関数を書け。また，そのヘッセ行列を書け。

--------


$N=5$のRosenbrock関数に対するヘッセ行列は，次のとおりである。

$$
H=
\begin{bmatrix}
1200x_0^2-400x_1+2 & -400x_0 & 0 & 0 & 0 \\
-400x_0 & 202+1200x_1^2-400x_2 & -400x_1 & 0 & 0\\
0 & -400x_1 & 202+1200x_2^2-400x_3 & -400x_2 & 0 \\
0 & 0 & -400x_2 & 202+1200x_3^2-400x_4 & 0  \\
0 & 0 & 0 & -400x_3 & 200 
\end{bmatrix}
$$

ヘッセ行列を用いてニュートンCG法で局所最適解を求めるプログラムは，次のとおりである。　

In [ ]:
from scipy.optimize import *
import numpy as np

In [ ]:
def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

In [ ]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'xtol': 1e-8, 'disp': True})

In [ ]:
print(res.x)

### ヘッセ行列とベクトルとの積を与える方法

大規模な問題を扱う際には，ヘッセ行列を保存したりその逆行列を求めたりするのは負荷が大きく現実的ではない。そのような場合は，ヘッセ行列を直接与えるのではなく，ヘッセ行列とベクトルとの積を与える方法が望ましい。

Rosenbrock関数のヘッセ行列と任意のベクトル($p$とする）との積は，次のとおりである。

$$
H(x)p= 
\begin{bmatrix}
\left(1200x_0^2-400x_1+2 \right)p_0-400x_0p_1 \\ 
\vdots \\
-400x_{i-1}p_{i-1}+\left(202+1200x_i^2-400x_{i+1}\right)p_i-400x_ip_{i+1} \\
\vdots \\ 
-400x_{N-2}p_{N-2}+200p_{N-1}
\end{bmatrix}
$$
このベクトルを与える関数は，次のとおりである。　


In [ ]:
def rosen_hess_p(x, p):
    x = np.asarray(x)
    Hp = np.zeros_like(x)
    Hp[0] = (1200*x[0]**2 - 400*x[1] + 2)*p[0] - 400*x[0]*p[1]
    Hp[1:-1] = -400*x[:-2]*p[:-2]+(202+1200*x[1:-1]**2-400*x[2:])*p[1:-1] \
               -400*x[1:-1]*p[2:]
    Hp[-1] = -400*x[-2]*p[-2] + 200*p[-1]
    return Hp

こうして定義した関数を用いてニュートンCG法にり最小解を求めるプログラムは，次のとおりである。

In [ ]:
res = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hessp=rosen_hess_p,
               options={'xtol': 1e-8, 'disp': True})

In [ ]:
res.x

--------

## 課題2

次の関数の最小化問題を，SciPyのニュートンCG法を用いて解け．

$$
f(x)=\sin\left(x_0x_1\right)+\left(x_0+x_1-1\right)^2 + \left(x_0-x_1+1\right)^4 
$$

その際，`minimize`の引数`jac=jacob`としてヤコビ行列を指定せよ．ここで，`jacob` はヤコビ行列を指定する関数である。

```
def func(x):
    return np.sin(........)......... ♯適切に書き換えてください。

```

--------


In [ ]:
def func(x):
    return np.sin(x[0]*x[1])+(x[0]+x[1]-1)**2+(x[0]-x[1]+1)**4

In [ ]:
def jacob(x):
    return np.array((4*(1+x[0]-x[1])**3+2*(x[0]+x[1]-1)+x[1]*np.cos(x[0]*x[1]),-4*(1+x[0]-x[1])**3+2*(x[0]+x[1]-1)+x[0]*np.cos(x[0]*x[1])))

In [ ]:
#ニュートンCG法で解くプログラム

--------------------


# 最適化技術実験 第2回　レポート
### 学生番号 000000 氏名 青学太郎

(ここから下に，レポートを作成してください。レポートを作成したら，「File」->「Save and Checkpoint」でこのjupyter notebookファイルを保存し，さらに，「File」->「Download as」->「Notebook(.ipynb)」を選択して手元のPCにダウンロードしてください．ダウンロードしたファイルの名前を「学生番号-氏名.ipynb」に変更して，CoursePowerから提出してください．ここで，ファイル名内の"学生番号"は1から始まる自分の学生番号に，"氏名"は自分の氏名に置き換えてください．）